# ЧМ, Домашняя работа №2-1

In [1]:
import numpy as np
from typing import Callable
from matplotlib import pyplot as plt


# Установка шрифта Times New Roman для всего графика
plt.rcParams['font.family'] = 'Times New Roman'
# Возможность использования TeX
plt.rcParams['text.usetex'] = True  # Предварительно следует установить latex и добваить его в PATH
# Добавление пакета babel для поддержки русского языка при использовании LaTeX в библиотеке Matplotlib.
plt.rcParams['text.latex.preamble'] = r"\usepackage[russian]{babel}"

# Номер студента в списке
N = 13
# Номер группы
n = 53


def f(tau: float) -> float:
    """
    Исходная функция в точке `tau`, на основе которой вычисляется сеточная фунцкия.
    """

    return (20+0.5*N)/(1 + (20 + 0.15*N)*(1 + 0.05*(53-n))*(tau-1)**2) 


def create_uniform_grid(a: float, b: float, n: int) -> np.linspace:
    """
    Возвращает равномерную сетку на промежутке `[a,b]` с кол-вом узлом равным `n`
    """

    return np.linspace(a,b,n)


def create_chebyshev_grid(a: float, b: float, n: int) -> np.array:
    """
    Возвращает Чебышевскую сетку на промежутке `[a,b]` с кол-вом узлом равным `n`
    """
     
    grid = [0] * n
    for j in range(n):
        grid[j] = (a+b)/2 - ((b-a)/2) * np.cos((2*j+1)*np.pi/(2*(n+1)))
    return np.array(grid)
    

def get_values_grid_function(grid: np.array, grid_function: Callable) -> np.array:
    """
    Возвращает список значений сеточной функции `grid_function` на сетке `grid`
    """

    values = []
    for tau in grid:
        values.append(grid_function(tau))
    return np.array(values)


def get_lagrange_polynomial_coefs(grid: np.array, values: np.array):
    """
    Функция для получения коэффециентов полинома Лагранжа
    """
    grid_size = len(grid)
    A = []
    for tau in grid:
        A.append([tau**n for n in range(grid_size)])

    b = values
    res =  np.linalg.solve(A,b)
    return np.array(res)


def L_0(tau_array : np.array, grid: np.array, values: np.array) -> np.array:
    """
    Функция, возвращающая значения полинома Лагранжа (считается через сумму произведений) на списке 
    значений `tau_array` на сетке `grid` и значениями сеточной функции `values`
    """

    grid_size = len(grid)

    def inner_L(tau):
        """
        Функция для вычисление полинома Лагранжа в точке `tau`
        """
        z=0
        for j in range(grid_size):
            p1=1; p2=1
            for i in range(grid_size):
                if i==j:
                    p1=p1*1; p2=p2*1   
                else: 
                    p1=p1*(tau-grid[i])
                    p2=p2*(grid[j]-grid[i])
            z=z+values[j]*p1/p2
        return z
    
    L_values = [inner_L(tau) for tau in tau_array]
    return np.array(L_values)


def L_1(tau_array : np.array, grid: np.array, values: np.array) -> np.array:
    """
    Функция, возвращающая значения полинома Лагранжа (коэф-ы определяются из решения СЛАУ) на списке 
    значений `tau_array` на сетке `grid` и значениями сеточной функции `values`
    """

    L_coefs = get_lagrange_polynomial_coefs(grid,values)

    def inner_L(tau):
        """
        Функция для вычисление полинома Лагранжа в точке `tau`
        """
        z = 0
        for i, coef in enumerate(L_coefs, start=0):
            z += coef * tau**i
        return z

    L_values = [inner_L(tau) for tau in tau_array]
    return np.array(L_values)


a = 0 # левая граница отрезка
b = 2 # правая граница отрезка

# РАВНОМЕРНАЯ СЕТКА 
grid = create_uniform_grid(a,b,41)   # для сеточной функции используем 41 равномерный узел
values = get_values_grid_function(grid, f)  # вычисление сеточной функции

# Построение точечного графика значений сеточной функции
plt.plot(grid,values, marker='o', color='orange',  linewidth=1.5, label=r"Сеточная функции $f( \tau )$")

# Построение полинома Лагранжа
grid = create_uniform_grid(a,b,21) # для полинома лагранжа используем 21 равнмоерный узел
values = get_values_grid_function(grid, f)
tau = np.linspace(a,b, 500)
L_values = L_1(tau, grid, values) # для вычисление через сумму произведений использовать L_0

print("РАВНОМЕРНАЯ СЕТКА:", end="\n\n")
print(f"Равномерная сетка на [{a}, {b}]:\n", grid, end="\n\n")
print("Значения сеточной функции на узлах сетки:\n", values, end="\n\n")
print("Коэффициенты полинома лагранжа:\n", get_lagrange_polynomial_coefs(grid, values), end="\n\n")

plt.xlabel(r"$\tau$")
plt.plot(tau, L_values, color="green", label=r"Полином Лагранжа $L_k( \tau ), k=21$")
plt.legend(loc="best")
plt.savefig("ugrid.pdf")
plt.show()
plt.close()


# ЧЕБЫШЕВСКАЯ СЕТКА
grid = create_uniform_grid(a,b,41)    # для сеточной функции используем 41 узел
values = get_values_grid_function(grid, f)

# Построение точечного графика значений сеточной функции
plt.plot(grid,values, marker='o', color='orange',  linewidth=2, label=r"Сеточная функции $f( \tau )$")

# Построение полинома Лагранжа
grid = create_chebyshev_grid(a,b,21) # для полинома лагранжа используем 21 чебышевский узел
values = get_values_grid_function(grid, f)
tau = np.linspace(a,2, 500)  
L_values = L_1(tau, grid, values) 

print("\nЧЕБЫШЕВСКАЯ СЕТКА:", end="\n\n")
print(f"Чебышевская сетка на [{a}, {b}]:\n", grid, end="\n\n")
print("Значения сеточной функции на узлах сетки:\n", values, end="\n\n")
print("Коэффициенты полинома лагранжа:\n", get_lagrange_polynomial_coefs(grid, values), end="\n\n")

plt.xlabel(r"$\tau$")
plt.plot(tau, L_values, color="green", label=r"Полином Лагранжа $L_k( \tau ), k=21$")
plt.legend(loc="best")
plt.savefig("chgrid.pdf")
plt.show()
plt.close()


РАВНОМЕРНАЯ СЕТКА:

Равномерная сетка на [0, 2]:
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2. ]

Значения сеточной функции на узлах сетки:
 [ 1.1546841   1.41111318  1.76103137  2.25426396  2.97685913  4.08477842
  5.87322695  8.90606621 14.11075612 21.7302173  26.5        21.7302173
 14.11075612  8.90606621  5.87322695  4.08477842  2.97685913  2.25426396
  1.76103137  1.41111318  1.1546841 ]

Коэффициенты полинома лагранжа:
 [ 1.15468410e+00 -7.56671946e+04  2.65847128e+06 -4.06762974e+07
  3.65594367e+08 -2.18599247e+09  9.30766467e+09 -2.94332974e+10
  7.10584330e+10 -1.33398122e+11  1.97058988e+11 -2.30619047e+11
  2.14312777e+11 -1.57815575e+11  9.14492318e+10 -4.11600808e+10
  1.40823260e+10 -3.53648723e+09  6.14301498e+08 -6.59100581e+07
  3.28958858e+06]



RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 432x288 with 1 Axes>